# PSO - Cornfield Vector 

    - Simular geograficamente o comportamento das espécies
    - Lembrança da melhor posição individual
    - Lembrança da melhor posição global

## Classe da função a ser minimizada

### Minimizar X^2 + Y^2

In [ ]:
class Utils:
    
    def function(position):
        '''
        Calculo da função
        '''
        position = np.array(position)
        result = sum(position**2)
            
        return result

## Classe das Partículas

In [ ]:
import random
import numpy as np

class Particle:
    '''
    Construtor
    '''
    def __init__(self, num_dimensions, bounds):
        self.num_dimensions = num_dimensions
        self.position = []
        self.velocity = []
        self.pbest_pos = [] # Melhor posição histórica da particula
        self.pbest_err = -1 # Melhor erro que a partícula teve
        self.err = -1 # Erro atual da partícula

        for i in range(0, self.num_dimensions):
            self.position.append(random.uniform(bounds[i][0], bounds[i][1]))
            self.velocity.append(random.uniform(bounds[i][0], bounds[i][1]))

    def evaluate(self, cost_function):
        '''
        Função que avalia e compara a proximidade de cada uma das partículas em relação ao objetivo
        '''
        self.err = cost_function(self.position)

        if self.err < self.pbest_err or self.pbest_err == -1:
            self.pbest_err = self.err
            self.pbest_pos = self.position

    def update_velocity(self, gbest_pos):
        '''
        Atualiza a velocidade com base em um "fator de craziness", o pbest e o gbest
        '''
        w = 0.5 #inércia
        c1 = 1
        c2 = 1

        for i in range(0, self.num_dimensions):
            r1 = random.random()
            r2 = random.random()

            vel_cognitive = c1 * r1 * (self.pbest_pos[i] - self.position[i])
            vel_social = c2 * r2 * (gbest_pos[i] - self.position[i])

            self.velocity[i] = w * self.velocity[i] + vel_cognitive + vel_social

    def update_position(self, bounds):
        '''
        Atualiza a posição de cada uma das partículas
        '''
        for i in range(0, self.num_dimensions):
            self.position[i] = self.position[i] + self.velocity[i]

            if self.position[i] < bounds[i][0]:
                self.position[i] = bounds[i][0]

            if self.position[i] > bounds[i][1]:
                self.position[i] = bounds[i][1]
            

## Classe PSO

In [ ]:
class PSO:

    def __init__(self, num_dimensions, cost_function, bounds, num_particles, max_iterations):
        '''
        Construtor
        '''
        gbest_err = -1
        gbest_pos = []
        
        PlotUtils.start_plot()
        
        swarm = []
        for i in range(0, num_particles):
            swarm.append(Particle(num_dimensions, bounds))

        i = 0
        while i < max_iterations:
            print(f"i = {i}\tgbest_pos = {gbest_pos}\tgbest_err = {gbest_err}")

            for j in range(0, num_particles):
                swarm[j].evaluate(cost_function)

                if swarm[j].err < gbest_err  or gbest_err == -1:
                    gbest_pos = list(swarm[j].position)
                    gbest_err = float(swarm[j].err)

            for j in range(0, num_particles):
                swarm[j].update_velocity(gbest_pos)
                swarm[j].update_position(bounds)

                PlotUtils.plot_particle(swarm[j])
            
            PlotUtils.plot_iteration(i)
            i += 1


        PlotUtils.save()
        print(f"i = {i}\tgbest_pos = {gbest_pos}\tgbest_err = {gbest_err}")

## Classe de Plot

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import glob
import os
import shutil

class PlotUtils:

    directory = "pso_plots"
    filename = 'pso_cornfield.gif'

    @staticmethod
    def start_plot():
        if os.path.exists(PlotUtils.directory):
            shutil.rmtree(PlotUtils.directory)
        if not os.path.exists(PlotUtils.directory):
            os.makedirs(PlotUtils.directory)

    @staticmethod
    def plot_particle(particle):
        plt.scatter(particle.position[0], particle.position[1])

    @staticmethod
    def plot_iteration(i):
        plt.title(f"PSO {i}")
        plt.xlim(-20, 20)
        plt.ylim(-20, 20)
        plt.xlabel('x[0]')
        plt.ylabel('x[1]')
        iteration = str(i).zfill(5)
        plt.savefig(f"pso_plots/iteration_{iteration}.png", facecolor = "white", dpi = 75)
        plt.close()

    @staticmethod
    def save():
        images = [Image.open(f) for f in sorted(glob.glob(PlotUtils.directory+"/*"))]
        img = images[0]
        img.save(fp=PlotUtils.filename, format='GIF', append_images=images, save_all=True, duration=200, loop=0)
        if os.path.exists(PlotUtils.directory):
            shutil.rmtree(PlotUtils.directory)

## Main

In [ ]:
if __name__ == "__main__":
    
    num_dimensions = 2
    cost_function = Utils.function
    input_bounds = [(-15, +15), (-15, +15)]
    num_particles = 100
    max_iterations = 25

    PSO(num_dimensions, cost_function, input_bounds, num_particles, max_iterations)